# Test formation of quasicrystal by projection of high-dimentional unit cells

In [2]:
%matplotlib inline
from neurotools.nlab import *
from edges_and_triangles import *

Loading nlab namespace
Could not locate the spectrum module, please install it
Multitaper methods will not work
Could not locate the spectrum module, please install it
Multitaper methods will not work


In [3]:
# Generate a collection of paths in M dimensions
# Use the unit vector projection to identify the point location
# TODO: we need a better datastructure for detecting identical points
# I don't know how to do this theoretically unless they occupy
# The same location in the higher-dimensional lattice?

# Hypothesis: two projections are identical if and only if their unit vectors agree? 
# This may speed things up?

# Still not fast enough. Let's grow the graph from a seed
# Using the high-dimensional representation
# Iteratively, but only adding points that are close to the plane?
# This *may* involve computing distance to a hyperplane. Yuck.
# Hmm.

# What if we test points by projecting into 2D and out again and seeing whether this operation is a fixed point?

In [4]:
# Dimensionality
M = 4
# Scale 
K = 10
# Maximum radius of pattern
R = 15
# Number of nearest-neighbor edge hops to compute when defining the lattice tiling
I = 50
# Unit vectors; make explicit no using complex this time
phases  = arange(M)*pi/M
units2d = np.float64([cos(phases),sin(phases)])

In [1]:
basis

NameError: name 'basis' is not defined

In [23]:
# To check whether a point is on the hyperplane, project from M -> 2 -> M?
# This is linear, combine
project2d = units2d.T.dot(units2d)
project2d.shape

# Define the M-D basis
basis = np.int32(np.eye(M))

# Construct an approximation of the M-D basis using the 2D unit vectors
basis2d = project2d.dot(basis)

# I think this is the orthogonal complement?
complement = basis - basis2d
ocomp = scipy.linalg.orth(complement)

# Check ocomp is full rank
print(det(ocomp))

# Check ocomp is unitary
print(ocomp.T.dot(ocomp))

# Check that ocomp and units2D are orthogonal This should be zero
print(units2d.dot(ocomp))

ocomp.shape

1.0
[[  1.00000000e+00  -1.79380389e-16  -1.21802426e-16   1.57009246e-16]
 [ -1.79380389e-16   1.00000000e+00  -3.52068201e-17  -1.57009246e-16]
 [ -1.21802426e-16  -3.52068201e-17   1.00000000e+00   4.26642159e-17]
 [  1.57009246e-16  -1.57009246e-16   4.26642159e-17   1.00000000e+00]]
[[ -7.07106781e-01  -7.07106781e-01   1.00000000e+00  -1.23869238e-16]
 [ -7.07106781e-01   7.07106781e-01   1.21168839e-16   1.00000000e+00]]


(4, 4)

In [24]:
# Don't use a grid, use a seed.
points = np.zeros((1,M),dtype=np.int32)
# Define units step vectors in all M dimensions
steps = np.concatenate([basis,-basis]).T

# Store located points in a set
hyperpoints = set(map(tuple,points))

method = 'hyperplane'

for iteration in range(I):
    # print(iteration)
    # Generate adjacent vectors
    adjacent = (points[...,None] + steps[None,:]).transpose((0,2,1))
    npt,nadj,ndim = adjacent.shape
    adjacent = adjacent.reshape((npt*nadj,ndim))
    
    # Remove existing vectors
    new = array(list(set(map(tuple,adjacent)) - hyperpoints))
    
    # Form projection onto 2D slice
    new2d = new.dot(units2d.T)
    
    # Remove outside radius
    distance = sqrt(sum(new2d**2,axis=1))
    keep = distance<R
    new2d = new2d[keep,:]

    if method=='hyperplane':
        # Depricated: using distance from hyperplane
        new = new[keep,:]
        projectednew = projectednew[keep,:]
        # Get distance
        distance = sqrt(sum(new.dot(ocomp)**2,axis=1))
        # Calculate errors incurred by projection
        # Points in this slice should have small error
        distance = sum((projectednew - new)**2,axis=1)**0.5
        mind = np.min(distance)    
        # print("Minimum error is",minerror)
        # Select points to add
        inslice = distance<=sqrt(M)#+1e-4
        # Add these points to hyperpoints
        new = new[inslice]
    if method=='quantize':
        # New method: quantizing the projection
        new = np.round(new2d.dot(units2d))
    
    hyperpoints |= set(map(tuple,new))
    # Use newfound points as seed for next iteration
    points = new
    if new.shape[0]<1:
        print("No new points found")
        break
    
points = units2d.dot(array(list(hyperpoints)).T)
distance = sqrt(sum(points**2,axis=0))
figure(figsize=(15,15))
scatter(*points,s=1)
force_aspect()

NameError: name 'projectednew' is not defined

# Complete tiling using Voronoi regions

In [ ]:
from scipy.spatial import Voronoi, voronoi_plot_2d
figure(figsize=(14,14))

vor = Voronoi(points.T,qhull_options="Qc")

v = vor.vertices
v = v[:,0]+1j*v[:,1]
v = array(v)
r = array(vor.regions)

ee = concatenate([array(list(zip(array(e)[arange(-1,len(e))],e))) for e in r])

v[abs(v)>R]=NaN

ok = (abs(diff(v[ee],1,1))<1).squeeze()
print(ok.shape,ee.shape)
plot_edges(v,ee[ok,:],color=BLACK,lw=0.5)

#xlim(-R,R)
#ylim(-R,R)
force_aspect()

# Complete tiling using Delaunay triangulation

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.tri as tri
import numpy as np
import math
pp = points
# Create the Triangulation; no triangles so Delaunay triangulation created.
triang = tri.Triangulation(pp[0],pp[1])
# Plot the triangulation.
plt.figure(figsize=(15,15))
plt.gca().set_aspect('equal')
plt.triplot(triang, 'bo-', lw=0.5,markersize=0,color=BLACK)
plt.title('triplot of Delaunay triangulation')
#xlim(-R,R)
#ylim(-R,R)

# Complete tiling using adjacency graph of hypercrystal

Edges exist if two points are separated by edit distance of 1. Neighborhood-expand existing point set to find them

In [ ]:
# Convert point set to array
hyperlist = list(hyperpoints)
x = np.int8(hyperlist)
# Create ID lookup table
hyperid = dict(zip(hyperpoints,range(len(hyperlist))))
# Generate adjacent vectors
adjacent = (x[...,None] + steps[None,:]).transpose((0,2,1))
npt,nadj,ndim = adjacent.shape
# Find all neighbots in the hyperpoints set
neighbors = [set(map(tuple,adj))&hyperpoints for adj in adjacent]
# Convert to indecies into point array
neighbor_id = [[hyperid[p] for p in nn] for nn in neighbors]

In [ ]:
# Render 
figure(figsize=(14,14))

v = array(points.T)
v = v[:,0]+1j*v[:,1]
v = array(v)

ee = array([(i,a) for (i,nn) in enumerate(neighbor_id) for a in nn])

#v[abs(v)>R]=NaN
plot_edges(v,ee,color=BLACK,lw=0.5)

#xlim(-R,R)
#ylim(-R,R)

In [ ]:
points.shape,len(hyperpoints)

In [ ]:
ee.shape